## Data Inspection

In [1]:
# import pandas as pd

# df = pd.read_csv('diabetes_012_health_indicators_BRFSS2015.csv')
# print(df.shape)
# print(df.columns)
# print(df.info())
# print(df.describe())


In [2]:
# # Binary Target
# df['Diabetes_binary'] = df['Diabetes_012'].apply(lambda x: 1 if x > 0 else 0)


In [3]:
# import matplotlib.pyplot as plt

# df['Diabetes_binary'].value_counts(normalize=True).plot(kind='bar')
# plt.xticks([0, 1], ['No Diabetes', 'Prediabetes/Diabetes'], rotation=0)
# plt.title('Class Distribution')
# plt.ylabel('Proportion')
# plt.show()


In [4]:
# import seaborn as sns

# correlations = df.corr()['Diabetes_binary'].sort_values(ascending=False)
# print(correlations.head(10))

# top_corr_features = correlations[1:11].index
# sns.heatmap(df[top_corr_features].corr(), annot=True, cmap='coolwarm')
# plt.title('Heatmap of Top Correlated Features')
# plt.show()


In [5]:
# # Correct numerical features
# num_features = ['BMI', 'PhysHlth', 'MentHlth', 'GenHlth']

# for feature in num_features:
#     sns.boxplot(x='Diabetes_binary', y=feature, data=df)
#     plt.title(f'{feature} vs Diabetes_binary')
#     plt.xticks([0, 1], ['No Diabetes', 'Prediabetes/Diabetes'])
#     plt.show()


In [6]:
# cat_features = ['HighBP', 'HighChol', 'Smoker', 'PhysActivity', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income']

# for feature in cat_features:
#     sns.countplot(x=feature, hue='Diabetes_binary', data=df)
#     plt.title(f'{feature} Distribution by Diabetes Status')
#     plt.legend(title='Diabetes', labels=['No', 'Yes'])
#     plt.show()


In [7]:
# from sklearn.feature_selection import SelectKBest, f_classif

# X = df.drop(columns=['Diabetes_012', 'Diabetes_binary'])
# y = df['Diabetes_binary']

# selector = SelectKBest(score_func=f_classif, k=10)
# selector.fit(X, y)

# scores = pd.DataFrame({
#     'Feature': X.columns,
#     'F-Score': selector.scores_
# }).sort_values(by='F-Score', ascending=False)

# print(scores)


In [8]:
# # BMI vs. Diabetes by HighBP & HighChol
# g = sns.FacetGrid(df, col='HighBP', row='HighChol', hue='Diabetes_binary', margin_titles=True)
# g.map_dataframe(sns.kdeplot, x='BMI', fill=True)
# g.add_legend()
# plt.show()


## Neural Network 2: 6-Layer with 8, 16, 21, 32 Neurons

---

In [3]:
# Produces the required model for all neural networks
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

def make_model(units):
    # Returns a compiled 6-layer neural network with an assigned amount of neurons
    model = Sequential([
        Input(shape=(21,)),        # 21 normalized features in
        Dense(units, activation='relu'),
        Dense(units, activation='relu'),
        Dense(units, activation='relu'),
        Dense(units, activation='relu'),
        Dense(units, activation='relu'),
        Dense(units, activation='relu'),
        Dense(3, activation='softmax')  # 3 output classes
    ])

    # Compile
    model.compile(
        optimizer=Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [4]:
# Cell 1: Load & Normalize the Data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 1. Load the Kaggle CSV (adjust path as needed)
df = pd.read_csv('diabetes_012_health_indicators_BRFSS2015.csv')

# 2. Split into features (X) and target (y)
X = df.drop('Diabetes_012', axis=1).values
y = df['Diabetes_012'].values

# 3. Train/test split (stratify to keep class proportions)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 4. Apply Min–Max normalization so all features lie in [0,1]
scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm  = scaler.transform(X_test)

# Now X_train_norm and X_test_norm are ready for your network.
print(X_train_norm.shape, y_train.shape)  # should be (202944, 21), (202944,)
print(X_test_norm.shape,  y_test.shape)   # should be (50736, 21), (50736,)
print(X_train_norm.min(), X_train_norm.max())  # should be 0.0, 1.0

(202944, 21) (202944,)
(50736, 21) (50736,)
0.0 1.0000000000000002


In [7]:
# Cell 2: 6 layer neural network with 8 neurons
#         Build, train & evaluate 8-unit model
import time 
# numpy as np
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

model_8_neurons = make_model(8)
model_8_neurons.summary()

start = time.time()
history_8 = model_8_neurons.fit(
    X_train_norm, y_train,
    epochs=20, batch_size=32,
    validation_split=0.1, verbose=1
)
train_time_8 = time.time() - start

y_pred_8 = model_8_neurons.predict(X_test_norm).argmax(axis=1)
mse_8 = mean_squared_error(y_test, y_pred_8)
r2_8  = r2_score(y_test, y_pred_8)
acc_8 = accuracy_score(y_test, y_pred_8)

print(f"8 neurons time to train: {train_time_8:.1f}s, MSE: {mse_8:.4f}, R²: {r2_8:.4f}, ACC: {acc_8:.4f}")


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                │ (None, 8)              │           176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 563 (2.20 KB)

 Trainable params: 563 (2.20 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 4s 584us/step - accuracy: 0.8397 - loss: 0.4739 - val_accuracy: 0.8482 - val_loss: 0.3977
Epoch 2/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 566us/step - accuracy: 0.8447 - loss: 0.4052 - val_accuracy: 0.8482 - val_loss: 0.3956
Epoch 3/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 509us/step - accuracy: 0.8476 - loss: 0.3991 - val_accuracy: 0.8475 - val_loss: 0.3949
Epoch 4/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 493us/step - accuracy: 0.8450 - loss: 0.4031 - val_accuracy: 0.8519 - val_loss: 0.3918
Epoch 5/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 496us/step - accuracy: 0.8491 - loss: 0.3969 - val_accuracy: 0.8511 - val_loss: 0.3909
Epoch 6/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 490us/step - accuracy: 0.8490 - loss: 0.3960 - val_accuracy: 0.8497 - val_loss: 0.3931
Epoch 7/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 496us/step - accuracy: 0.8482 - loss: 0.3957 - val_accuracy: 0.8505 - val_loss: 0.3906
Epoch 8/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 521us/step - accuracy: 0.8472 -

In [10]:
# Cell 2: 6 layer neural network with 16 neurons
#         Build, train & evaluate 8-unit model
import time 
# numpy as np
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

model_16_neurons = make_model(16)
model_16_neurons.summary()

start = time.time()
history_16 = model_16_neurons.fit(
    X_train_norm, y_train,
    epochs=20, batch_size=32,
    validation_split=0.1, verbose=1
)
train_time_16 = time.time() - start

y_pred_16 = model_16_neurons.predict(X_test_norm).argmax(axis=1)
mse_16 = mean_squared_error(y_test, y_pred_16)
r2_16  = r2_score(y_test, y_pred_16)
acc_16 = accuracy_score(y_test, y_pred_16)

print(f"16 neurons time to train: {train_time_16:.1f}s, MSE: {mse_16:.4f}, R²: {r2_16:.4f}, ACC: {acc_16:.4f}")


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_35 (Dense)                │ (None, 16)             │           352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,763 (6.89 KB)

 Trainable params: 1,763 (6.89 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 4s 569us/step - accuracy: 0.8336 - loss: 0.4383 - val_accuracy: 0.8490 - val_loss: 0.3966
Epoch 2/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 516us/step - accuracy: 0.8456 - loss: 0.4002 - val_accuracy: 0.8500 - val_loss: 0.3982
Epoch 3/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 516us/step - accuracy: 0.8463 - loss: 0.4002 - val_accuracy: 0.8499 - val_loss: 0.3939
Epoch 4/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 514us/step - accuracy: 0.8483 - loss: 0.3969 - val_accuracy: 0.8506 - val_loss: 0.3905
Epoch 5/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 519us/step - accuracy: 0.8475 - loss: 0.3981 - val_accuracy: 0.8504 - val_loss: 0.3898
Epoch 6/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 514us/step - accuracy: 0.8474 - loss: 0.3986 - val_accuracy: 0.8507 - val_loss: 0.3903
Epoch 7/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 524us/step - accuracy: 0.8485 - loss: 0.3948 - val_accuracy: 0.8511 - val_loss: 0.3917
Epoch 8/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 511us/step - accuracy: 0.8468 -

In [ ]:
# Cell 2: 6 layer neural network with 21 neurons
#         Build, train & evaluate 8-unit model
import time 
# numpy as np
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

model_21_neurons = make_model(21)
model_21_neurons.summary()

start = time.time()
history_21 = model_21_neurons.fit(
    X_train_norm, y_train,
    epochs=20, batch_size=32,
    validation_split=0.1, verbose=1
)
train_time_21 = time.time() - start

y_pred_21 = model_16_neurons.predict(X_test_norm).argmax(axis=1)
mse_21 = mean_squared_error(y_test, y_pred_16)
r2_21  = r2_score(y_test, y_pred_16)
acc_21 = accuracy_score(y_test, y_pred_16)

print(f"21 neurons time to train: {train_time_21:.1f}s, MSE: {mse_21:.4f}, R²: {r2_21:.4f}, ACC: {acc_21:.4f}")


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_42 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 21)             │           462 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 3)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,838 (11.09 KB)

 Trainable params: 2,838 (11.09 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 4s 612us/step - accuracy: 0.8415 - loss: 0.4385 - val_accuracy: 0.8490 - val_loss: 0.3969
Epoch 2/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 553us/step - accuracy: 0.8458 - loss: 0.4024 - val_accuracy: 0.8504 - val_loss: 0.3931
Epoch 3/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 573us/step - accuracy: 0.8458 - loss: 0.4009 - val_accuracy: 0.8496 - val_loss: 0.3965
Epoch 4/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 570us/step - accuracy: 0.8475 - loss: 0.3983 - val_accuracy: 0.8521 - val_loss: 0.3906
Epoch 5/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 559us/step - accuracy: 0.8477 - loss: 0.3977 - val_accuracy: 0.8508 - val_loss: 0.3911
Epoch 6/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 542us/step - accuracy: 0.8508 - loss: 0.3929 - val_accuracy: 0.8516 - val_loss: 0.3913
Epoch 7/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 535us/step - accuracy: 0.8478 - loss: 0.3987 - val_accuracy: 0.8510 - val_loss: 0.3913
Epoch 8/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 519us/step - accuracy: 0.8506 -

In [12]:
# Cell 2: 6 layer neural network with 32 neurons
#         Build, train & evaluate 8-unit model
import time 
# numpy as np
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

model_32_neurons = make_model(32)
model_32_neurons.summary()

start = time.time()
history_32 = model_32_neurons.fit(
    X_train_norm, y_train,
    epochs=20, batch_size=32,
    validation_split=0.1, verbose=1
)
train_time_32 = time.time() - start

y_pred_32 = model_32_neurons.predict(X_test_norm).argmax(axis=1)
mse_32 = mean_squared_error(y_test, y_pred_32)
r2_32  = r2_score(y_test, y_pred_32)
acc_32 = accuracy_score(y_test, y_pred_32)

print(f"32 neurons time to train: {train_time_32:.1f}s, MSE: {mse_32:.4f}, R²: {r2_32:.4f}, ACC: {acc_32:.4f}")


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_49 (Dense)                │ (None, 32)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,083 (23.76 KB)

 Trainable params: 6,083 (23.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 4s 594us/step - accuracy: 0.8399 - loss: 0.4333 - val_accuracy: 0.8499 - val_loss: 0.3937
Epoch 2/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 534us/step - accuracy: 0.8464 - loss: 0.3998 - val_accuracy: 0.8507 - val_loss: 0.3918
Epoch 3/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 512us/step - accuracy: 0.8474 - loss: 0.4008 - val_accuracy: 0.8505 - val_loss: 0.3952
Epoch 4/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 515us/step - accuracy: 0.8471 - loss: 0.3973 - val_accuracy: 0.8502 - val_loss: 0.3905
Epoch 5/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 522us/step - accuracy: 0.8479 - loss: 0.3974 - val_accuracy: 0.8509 - val_loss: 0.3925
Epoch 6/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 511us/step - accuracy: 0.8485 - loss: 0.3967 - val_accuracy: 0.8512 - val_loss: 0.3913
Epoch 7/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 518us/step - accuracy: 0.8490 - loss: 0.3970 - val_accuracy: 0.8508 - val_loss: 0.3900
Epoch 8/20
5708/5708 ━━━━━━━━━━━━━━━━━━━━ 3s 520us/step - accuracy: 0.8485 -

In [15]:
# Table of results
import pandas as pd

results = [
    (8,  train_time_8,  mse_8,  r2_8,  acc_8),
    (16, train_time_16, mse_16, r2_16, acc_16),
    (21, train_time_21, mse_21, r2_21, acc_21),
    (32, train_time_32, mse_32, r2_32, acc_32),
]
df = pd.DataFrame(results, columns=['Units','Time(s)','MSE','R²','Acc'])
display(df)


,Units,Time(s),MSE,R²,Acc
0,8,61.389762,0.552152,-0.132824,0.848273
1,16,64.077474,0.551995,-0.132500,0.848313
2,21,66.338796,0.551995,-0.132500,0.848313
3,32,65.174513,0.553650,-0.135897,0.847899
